# E-Commerce Business Analytics - Exploratory Data Analysis

This notebook provides a comprehensive analysis of e-commerce business performance, focusing on revenue trends, product performance, geographic distribution, and customer experience metrics.

**Analysis Period**: Configurable by year and month  
**Last Updated**: 2025-10-03

## Table of Contents

1. [Introduction and Business Objectives](#1-introduction-and-business-objectives)
2. [Data Loading and Configuration](#2-data-loading-and-configuration)
3. [Data Dictionary](#3-data-dictionary)
4. [Data Preparation and Transformation](#4-data-preparation-and-transformation)
5. [Business Metrics Analysis](#5-business-metrics-analysis)
   - 5.1 [Revenue Analysis](#51-revenue-analysis)
   - 5.2 [Product Performance Analysis](#52-product-performance-analysis)
   - 5.3 [Geographic Performance Analysis](#53-geographic-performance-analysis)
   - 5.4 [Customer Experience Analysis](#54-customer-experience-analysis)
6. [Summary and Key Insights](#6-summary-and-key-insights)

## 1. Introduction and Business Objectives

### Business Objectives

This analysis aims to answer key business questions:

- **Revenue Performance**: How is revenue trending? What is the year-over-year growth?
- **Product Strategy**: Which product categories are driving the most revenue?
- **Geographic Expansion**: Which states represent our strongest markets?
- **Customer Satisfaction**: How satisfied are our customers? How does delivery speed impact satisfaction?
- **Operational Efficiency**: What is our average delivery time and how can we improve?

### Methodology

The analysis uses a modular approach with custom Python modules for data loading and metrics calculation, ensuring reproducibility and code reusability.

## 2. Data Loading and Configuration

Configure the analysis parameters below to analyze different time periods.

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import warnings

# Import custom modules
from data_loader import load_and_process_data
from business_metrics import BusinessMetricsCalculator, MetricsVisualizer

# Configure display settings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [ ]:
# Analysis Configuration
# Modify these parameters to analyze different time periods

ANALYSIS_YEAR = 2023           # Primary year to analyze
COMPARISON_YEAR = 2022         # Year to compare against (for YoY metrics)
ANALYSIS_MONTH = None          # Specific month (1-12) or None for full year
DATA_PATH = 'ecommerce_data/'  # Path to data files

print("Analysis Configuration:")
print(f"  Primary Period: {ANALYSIS_YEAR}" + (f"-{ANALYSIS_MONTH:02d}" if ANALYSIS_MONTH else ""))
print(f"  Comparison Period: {COMPARISON_YEAR}")
print(f"  Data Path: {DATA_PATH}")

In [ ]:
# Load and process all data
print("Loading data...")
loader, processed_data = load_and_process_data(DATA_PATH)

# Display data summary
summary = loader.get_data_summary()
print("\nData Summary:")
print(f"  Total Orders: {summary.get('total_orders', 'N/A'):,}")
print(f"  Date Range: {summary.get('date_range', {}).get('start', 'N/A')} to {summary.get('date_range', {}).get('end', 'N/A')}")
print(f"  Years Available: {summary.get('years_available', 'N/A')}")
print(f"  Total Products: {summary.get('total_products', 'N/A'):,}")
print(f"  Product Categories: {summary.get('product_categories', 'N/A')}")
print(f"  States: {summary.get('states', 'N/A')}")
print("\nData loaded successfully!")

## 3. Data Dictionary

### Key Tables and Columns

**Orders Table**
- `order_id`: Unique identifier for each order
- `customer_id`: Unique identifier for each customer
- `order_status`: Current status (delivered, shipped, canceled, pending, processing, returned)
- `order_purchase_timestamp`: When the order was placed
- `order_delivered_customer_date`: When the order was delivered to customer

**Order Items Table**
- `order_id`: Links to orders table
- `product_id`: Links to products table
- `price`: Product price (excludes freight)
- `freight_value`: Shipping cost

**Products Table**
- `product_id`: Unique identifier for each product
- `product_category_name`: Product category

**Customers Table**
- `customer_id`: Unique identifier for each customer
- `customer_state`: US state abbreviation
- `customer_city`: City name

**Reviews Table**
- `order_id`: Links to orders table
- `review_score`: Customer rating (1-5 scale, 5 being best)

### Calculated Metrics

- **Revenue**: Sum of product prices (excludes freight in totals)
- **Average Order Value (AOV)**: Total revenue divided by number of unique orders
- **Delivery Days**: Days between order purchase and customer delivery
- **Delivery Speed Categories**:
  - Fast: 1-3 days
  - Medium: 4-7 days
  - Slow: 8+ days
- **High Satisfaction**: Review score >= 4

## 4. Data Preparation and Transformation

Create filtered datasets for the analysis period and comparison period.

In [ ]:
# Create sales dataset for analysis period
print(f"Creating dataset for {ANALYSIS_YEAR}" + (f" Month {ANALYSIS_MONTH}" if ANALYSIS_MONTH else " (Full Year)") + "...")
sales_current = loader.create_sales_dataset(
    year_filter=ANALYSIS_YEAR,
    month_filter=ANALYSIS_MONTH,
    status_filter='delivered'
)

print(f"  Records: {len(sales_current):,}")
print(f"  Orders: {sales_current['order_id'].nunique():,}")
print(f"  Customers: {sales_current['customer_id'].nunique():,}")

# Create comparison dataset
print(f"\nCreating dataset for {COMPARISON_YEAR}" + (f" Month {ANALYSIS_MONTH}" if ANALYSIS_MONTH else " (Full Year)") + "...")
sales_comparison = loader.create_sales_dataset(
    year_filter=COMPARISON_YEAR,
    month_filter=ANALYSIS_MONTH,
    status_filter='delivered'
)

print(f"  Records: {len(sales_comparison):,}")
print(f"  Orders: {sales_comparison['order_id'].nunique():,}")
print(f"  Customers: {sales_comparison['customer_id'].nunique():,}")

In [ ]:
# Preview the sales dataset
print(f"Sample of {ANALYSIS_YEAR} Sales Data:")
display(sales_current.head())

print("\nData Types:")
display(sales_current.dtypes)

## 5. Business Metrics Analysis

Comprehensive analysis of business performance across multiple dimensions.

### 5.1 Revenue Analysis

Analyze overall revenue performance, trends, and year-over-year growth.

In [ ]:
# Calculate revenue metrics
calculator = BusinessMetricsCalculator(sales_current)
revenue_metrics = calculator.calculate_revenue_metrics(comparison_data=sales_comparison)

print(f"Revenue Metrics for {ANALYSIS_YEAR}:")
print("=" * 50)
print(f"Total Revenue: ${revenue_metrics['total_revenue']:,.2f}")
print(f"Total Orders: {revenue_metrics['total_orders']:,}")
print(f"Total Items Sold: {revenue_metrics['total_items']:,}")
print(f"\nAverage Order Value: ${revenue_metrics['average_order_value']:,.2f}")
print(f"Median Order Value: ${revenue_metrics['median_order_value']:,.2f}")
print(f"Average Items per Order: {revenue_metrics['avg_items_per_order']:.2f}")

if revenue_metrics.get('revenue_growth_pct') is not None:
    print(f"\nYear-over-Year Comparison vs {COMPARISON_YEAR}:")
    print("=" * 50)
    print(f"Revenue Growth: {revenue_metrics['revenue_growth_pct']:+.2f}%")
    print(f"Order Growth: {revenue_metrics['order_growth_pct']:+.2f}%")
    print(f"AOV Growth: {revenue_metrics['aov_growth_pct']:+.2f}%")

In [ ]:
# Calculate monthly trends (only for full year analysis)
if ANALYSIS_MONTH is None:
    monthly_trends = calculator.calculate_monthly_trends()
    
    print(f"\nMonthly Trends for {ANALYSIS_YEAR}:")
    print("=" * 80)
    
    # Format the display
    display_df = monthly_trends.copy()
    display_df['revenue'] = display_df['revenue'].apply(lambda x: f"${x:,.2f}")
    display_df['avg_order_value'] = display_df['avg_order_value'].apply(lambda x: f"${x:,.2f}")
    display_df['revenue_growth_pct'] = display_df['revenue_growth_pct'].apply(
        lambda x: f"{x:+.2f}%" if pd.notna(x) else "N/A"
    )
    display_df['orders_growth_pct'] = display_df['orders_growth_pct'].apply(
        lambda x: f"{x:+.2f}%" if pd.notna(x) else "N/A"
    )
    
    display(display_df)
    
    # Calculate average monthly growth
    avg_monthly_growth = monthly_trends['revenue_growth_pct'].mean()
    print(f"\nAverage Month-over-Month Revenue Growth: {avg_monthly_growth:.2f}%")
else:
    print("\nMonthly trends analysis is only available for full year analysis.")
    print(f"Current configuration is analyzing a specific month: {ANALYSIS_YEAR}-{ANALYSIS_MONTH:02d}")

In [ ]:
# Visualize revenue trend (only for full year)
if ANALYSIS_MONTH is None:
    visualizer = MetricsVisualizer()
    fig = visualizer.plot_revenue_trend(
        monthly_trends,
        title=f"Monthly Revenue Trend - {ANALYSIS_YEAR}"
    )
    plt.show()
else:
    print("Revenue trend visualization is only available for full year analysis.")

### 5.2 Product Performance Analysis

Identify top-performing product categories and their contribution to overall revenue.

In [ ]:
# Analyze product performance
product_performance = calculator.analyze_product_performance()

print(f"Product Category Performance for {ANALYSIS_YEAR}:")
print("=" * 80)

# Format for display
display_df = product_performance.copy()
display_df['revenue'] = display_df['revenue'].apply(lambda x: f"${x:,.2f}")
display_df['revenue_share_pct'] = display_df['revenue_share_pct'].apply(lambda x: f"{x:.2f}%")
display_df['avg_order_value'] = display_df['avg_order_value'].apply(lambda x: f"${x:,.2f}")

display(display_df)

# Top 3 categories
print("\nTop 3 Product Categories by Revenue:")
for idx, row in product_performance.head(3).iterrows():
    print(f"  {idx+1}. {row['category']}: ${row['revenue']:,.2f} ({row['revenue_share_pct']:.2f}% of total)")

In [ ]:
# Visualize product category performance
visualizer = MetricsVisualizer()
fig = visualizer.plot_category_performance(
    product_performance,
    top_n=10,
    title=f"Top 10 Product Categories by Revenue - {ANALYSIS_YEAR}"
)
plt.show()

### 5.3 Geographic Performance Analysis

Analyze revenue distribution across different states to identify key markets.

In [ ]:
# Analyze geographic performance
geo_performance = calculator.analyze_geographic_performance()

print(f"Geographic Performance for {ANALYSIS_YEAR}:")
print("=" * 80)

# Show top 10 states
display_df = geo_performance.head(10).copy()
display_df['revenue'] = display_df['revenue'].apply(lambda x: f"${x:,.2f}")
display_df['revenue_share_pct'] = display_df['revenue_share_pct'].apply(lambda x: f"{x:.2f}%")
display_df['avg_order_value'] = display_df['avg_order_value'].apply(lambda x: f"${x:,.2f}")

print("\nTop 10 States by Revenue:")
display(display_df)

# Summary stats
print(f"\nTop 5 states account for {geo_performance.head(5)['revenue_share_pct'].sum():.2f}% of total revenue")

In [ ]:
# Visualize geographic distribution
visualizer = MetricsVisualizer()
fig = visualizer.plot_geographic_map(
    geo_performance,
    title=f"Revenue by State - {ANALYSIS_YEAR}"
)
fig.show()

### 5.4 Customer Experience Analysis

Evaluate customer satisfaction through review scores and delivery performance metrics.

In [ ]:
# Analyze customer satisfaction
satisfaction_metrics = calculator.analyze_customer_satisfaction()

print(f"Customer Satisfaction Metrics for {ANALYSIS_YEAR}:")
print("=" * 50)
print(f"Total Reviews: {satisfaction_metrics['total_reviews']:,}")
print(f"Average Review Score: {satisfaction_metrics['avg_review_score']:.2f} / 5.00")
print(f"Median Review Score: {satisfaction_metrics['median_review_score']:.0f} / 5")
print(f"High Satisfaction Rate (4-5 stars): {satisfaction_metrics['high_satisfaction_pct']:.2f}%")

print("\nReview Score Distribution:")
for score in sorted(satisfaction_metrics['score_distribution'].keys()):
    pct = satisfaction_metrics['score_distribution'][score] * 100
    print(f"  {int(score)} stars: {pct:.2f}%")

In [ ]:
# Visualize review score distribution
visualizer = MetricsVisualizer()
fig = visualizer.plot_review_distribution(
    sales_current,
    title=f"Review Score Distribution - {ANALYSIS_YEAR}"
)
plt.show()

In [ ]:
# Analyze delivery performance
delivery_metrics = calculator.analyze_delivery_performance()

print(f"\nDelivery Performance Metrics for {ANALYSIS_YEAR}:")
print("=" * 50)
print(f"Average Delivery Time: {delivery_metrics['avg_delivery_days']:.1f} days")
print(f"Median Delivery Time: {delivery_metrics['median_delivery_days']:.0f} days")

print("\nDelivery Speed Distribution:")
for category, pct in delivery_metrics['delivery_category_distribution'].items():
    print(f"  {category}: {pct*100:.2f}%")

if 'avg_review_by_delivery_speed' in delivery_metrics:
    print("\nAverage Review Score by Delivery Speed:")
    for category, score in delivery_metrics['avg_review_by_delivery_speed'].items():
        print(f"  {category}: {score:.2f} / 5.00")

In [ ]:
# Visualize delivery performance impact on satisfaction
visualizer = MetricsVisualizer()
fig = visualizer.plot_delivery_performance(
    sales_current,
    title=f"Average Review Score by Delivery Speed - {ANALYSIS_YEAR}"
)
plt.show()

## 6. Summary and Key Insights

This section provides a comprehensive summary of the analysis findings.

In [ ]:
# Generate comprehensive summary
print(f"EXECUTIVE SUMMARY - {ANALYSIS_YEAR}")
print("=" * 80)

# Revenue Summary
print("\nREVENUE PERFORMANCE")
print("-" * 40)
print(f"Total Revenue: ${revenue_metrics['total_revenue']:,.2f}")
if revenue_metrics.get('revenue_growth_pct') is not None:
    growth_direction = "increase" if revenue_metrics['revenue_growth_pct'] > 0 else "decrease"
    print(f"YoY Change: {abs(revenue_metrics['revenue_growth_pct']):.2f}% {growth_direction} vs {COMPARISON_YEAR}")
print(f"Total Orders: {revenue_metrics['total_orders']:,}")
print(f"Average Order Value: ${revenue_metrics['average_order_value']:,.2f}")

# Product Performance Summary
print("\nPRODUCT PERFORMANCE")
print("-" * 40)
top_category = product_performance.iloc[0]
print(f"Top Category: {top_category['category']}")
print(f"Top Category Revenue: ${top_category['revenue']:,.2f} ({top_category['revenue_share_pct']:.2f}% of total)")
print(f"Total Categories: {len(product_performance)}")

# Geographic Performance Summary
print("\nGEOGRAPHIC PERFORMANCE")
print("-" * 40)
top_state = geo_performance.iloc[0]
print(f"Top State: {top_state['state']}")
print(f"Top State Revenue: ${top_state['revenue']:,.2f} ({top_state['revenue_share_pct']:.2f}% of total)")
print(f"Total States: {len(geo_performance)}")

# Customer Experience Summary
print("\nCUSTOMER EXPERIENCE")
print("-" * 40)
print(f"Average Review Score: {satisfaction_metrics['avg_review_score']:.2f} / 5.00")
print(f"High Satisfaction Rate: {satisfaction_metrics['high_satisfaction_pct']:.2f}%")
print(f"Average Delivery Time: {delivery_metrics['avg_delivery_days']:.1f} days")

print("\n" + "=" * 80)

### Key Observations

Based on the analysis above, key insights include:

**Revenue Trends**
- Review the revenue growth rate and identify any significant changes from the previous year
- Monitor month-over-month trends to identify seasonal patterns

**Product Strategy**
- Focus marketing and inventory efforts on top-performing categories
- Identify underperforming categories for potential improvement or discontinuation

**Geographic Expansion**
- Top-performing states represent key markets for targeted campaigns
- Lower-performing states may present growth opportunities

**Customer Experience**
- Faster delivery times are associated with higher review scores
- Current satisfaction levels indicate customer retention potential

### Next Steps

1. Deep-dive analysis on specific underperforming categories
2. Investigate opportunities to reduce delivery times
3. Conduct cohort analysis to understand customer lifetime value
4. Analyze pricing strategies by category and geography